In [20]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
from ast import literal_eval
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
import string

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [21]:
# Import data
tsd_train = pd.read_csv('gdrive/MyDrive/CSI4900-Project/data/tsd_train.csv')
tsd_test = pd.read_csv('gdrive/MyDrive/CSI4900-Project/data/tsd_test.csv')
tsd_trial = pd.read_csv('gdrive/MyDrive/CSI4900-Project/data/tsd_trial.csv')

# Convert spans from string to list
tsd_train['spans'] = tsd_train['spans'].apply(literal_eval)
tsd_test['spans'] = tsd_test['spans'].apply(literal_eval)
tsd_trial['spans'] = tsd_trial['spans'].apply(literal_eval)

tsd_train.head(10)

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."
5,[],"But, but, but, is NOT a defense. It's not eve..."
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","Please people, stop using these silly, stupid ..."
7,"[0, 1, 2, 3]",Dumb.
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",Obamacare is on it's last gasping breaths. Y...
9,"[32, 33, 34, 35, 36, 37, 38, 39]",CROOKED Trump = GUILTY as hell.\npathetic


In [22]:
tsd_test.head(10)

,spans,text
0,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...",That's right. They are not normal. And I am st...
1,"[81, 82, 83, 84, 85, 86]","""Watch people die from taking away their healt..."
2,[],tens years ago i contacted the PDR and suggest...
3,[],The parallels between the ANC and the Sicilian...
4,[],Intel Community: ‘How can we work for a Presid...
5,"[129, 130, 131, 132, 133, 134]",I live in an area that saw major flood damage ...
6,"[35, 36, 37, 38, 39, 40, 41, 42, 43]",Elaine Marie Jeffers sounds like a sociopath 😕.
7,"[55, 56, 57, 58, 59, 60, 61, 62, 114, 115, 116...",A rock has more mental horsepower than the ind...
8,"[37, 38, 39, 40, 41, 158, 159, 160, 161, 204, ...","Ha ha, HILLARY LOST. All you hillary fools nee..."
9,"[94, 95, 96, 97]",Yes....it is to be expected....oddly enough it...


In [23]:
# Lowercase all characters
tsd_train['text'] = tsd_train['text'].apply(lambda x: x.lower())
tsd_test['text'] = tsd_test['text'].apply(lambda x: x.lower())
tsd_trial['text'] = tsd_trial['text'].apply(lambda x: x.lower())

# Replace \n by space, so the index won't be effected
# Replace emojis by ?
tsd_train['text'] = tsd_train['text'].apply(lambda x: x.replace('\n', ' '))
tsd_train['text'] = tsd_train['text'].apply(lambda x: x.encode('ascii', 'replace').decode('ascii'))

tsd_test['text'] = tsd_test['text'].apply(lambda x: x.replace('\n', ' '))
tsd_test['text'] = tsd_test['text'].apply(lambda x: x.encode('ascii', 'replace').decode('ascii'))

tsd_trial['text'] = tsd_trial['text'].apply(lambda x: x.replace('\n', ' '))
tsd_trial['text'] = tsd_trial['text'].apply(lambda x: x.encode('ascii', 'replace').decode('ascii'))

# Remove punctuations
def remove_punctuation(text):
    punctuationfree = []
    for i in text:
        if i not in string.punctuation:
            punctuationfree.append(i)
        else:
            punctuationfree.append(' ')
    if punctuationfree[-1] == ' ':
        punctuationfree = punctuationfree[:-1]
    punctuationfree = ''.join(punctuationfree)
    return punctuationfree

tsd_train['text'] = tsd_train['text'].apply(lambda x: remove_punctuation(x))
tsd_trial['text'] = tsd_trial['text'].apply(lambda x: remove_punctuation(x))
tsd_test['text'] = tsd_test['text'].apply(lambda x: remove_punctuation(x))

tsd_train.head(10)

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]",i am 56 years old i am not your fucking junio...
2,"[0, 1, 2, 3]",damn a whole family sad indeed
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",what a knucklehead how can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]",who do you think should do the killing any...
5,[],but but but is not a defense it s not eve...
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...",please people stop using these silly stupid ...
7,"[0, 1, 2, 3]",dumb
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",obamacare is on it s last gasping breaths y...
9,"[32, 33, 34, 35, 36, 37, 38, 39]",crooked trump guilty as hell pathetic


In [24]:
tsd_test['text'][9]

'yes    it is to be expected    oddly enough it happens every time he opens his mouth and says dumb things'

In [9]:
def find_toxic_words(span, text):
    i = 0
    token = []
    a = 0

    word = []

    while (i < (len(span) - 1)):
        if (span[i] != (span[i+1]-1)):
            token.append(span[a:(i+1)])
            a = i + 1
        elif i == (len(span) - 2):
            token.append(span[a:i+2])

        i = i + 1

    for t in token:
        word.append(text[t[0]:(t[len(t)-1])+1])

    return word

def BIO_tagger(span, text, get_span=0):
    tknzr2 = TweetTokenizer()
    tmp_tokens = tknzr2.tokenize(text)
    tokens = []
    token_span = []
    for each_token in tmp_tokens:
        if each_token not in string.punctuation:
            tokens.append(each_token)
            if get_span == 1:
                if len(token_span) != 0:
                    tmp = text[token_span[len(token_span)-1][-1]+1:]
                    start = tmp.find(each_token) + token_span[len(token_span)-1][-1]+1
                    token_span.append(list(range(start, start + len(each_token))))
                else:
                    token_span.append(list(range(0, len(each_token))))

    sequence = []
    for each in tokens:
        sequence.append([each, 'O'])

    toxic_words = find_toxic_words(span, text)

    toxic_tokens = []
    for i in range(0, len(tokens)):
        for each_toxic in toxic_words:
            t = tknzr2.tokenize(each_toxic)
            if tokens[i] in t:
                if tokens[i] == t[0]:
                    sequence[i][1] = 'B'
                elif i != 0 and (sequence[i-1][1] == 'B' or sequence[i-1][1] == 'I'):
                    sequence[i][1] = 'I'

    return sequence, toxic_words, token_span

# Build a new dataframe to store tokens and tags
def encode(df, get_span=0):
    result_sequence  = []
    result_toxic_words = []
    result_token_span = []

    for index, row in df.iterrows():
        sequence, toxic_words, token_span = BIO_tagger(row['spans'], row['text'], get_span)
        result_sequence.append(sequence)
        result_toxic_words.append(toxic_words)
        result_token_span.append(token_span)

    df['sequence'] = result_sequence
    df['toxic_word'] = result_toxic_words
    if get_span == 1:
        df['token_span'] = result_token_span

encode(tsd_train)
encode(tsd_trial)
encode(tsd_test, 1)

tsd_train.head(10)

,spans,text,sequence,toxic_word
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",another violent and aggressive immigrant killi...,"[[another, O], [violent, B], [and, I], [aggres...",[violent and aggressive immigrant]
1,"[33, 34, 35, 36, 37, 38, 39]",i am 56 years old i am not your fucking junio...,"[[i, O], [am, O], [56, O], [years, O], [old, O...",[fucking]
2,"[0, 1, 2, 3]",damn a whole family sad indeed,"[[damn, B], [a, O], [whole, O], [family, O], [...",[damn]
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",what a knucklehead how can anyone not know th...,"[[what, O], [a, O], [knucklehead, B], [how, O]...",[knucklehead]
4,"[32, 33, 34, 35, 36, 37, 38]",who do you think should do the killing any...,"[[who, O], [do, O], [you, O], [think, O], [sho...",[killing]
5,[],but but but is not a defense it s not eve...,"[[but, O], [but, O], [but, O], [is, O], [not, ...",[]
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...",please people stop using these silly stupid ...,"[[please, O], [people, O], [stop, O], [using, ...","[stupid, emoticons]"
7,"[0, 1, 2, 3]",dumb,"[[dumb, B]]",[dumb]
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",obamacare is on it s last gasping breaths y...,"[[obamacare, O], [is, O], [on, O], [it, O], [s...","[idiots, stupid]"
9,"[32, 33, 34, 35, 36, 37, 38, 39]",crooked trump guilty as hell pathetic,"[[crooked, O], [trump, O], [guilty, O], [as, O...",[pathetic]


In [10]:
tsd_test.head(10)

,spans,text,sequence,toxic_word,token_span
0,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...",that s right they are not normal and i am st...,"[[that, O], [s, O], [right, O], [they, O], [ar...","[abnormal, sexist rubbish]","[[0, 1, 2, 3], [5], [7, 8, 9, 10, 11], [14, 15..."
1,"[81, 82, 83, 84, 85, 86]",watch people die from taking away their healt...,"[[watch, O], [people, O], [die, O], [from, O],...",[stupid],"[[0, 1, 2, 3, 4], [7, 8, 9, 10, 11, 12], [14, ..."
2,[],tens years ago i contacted the pdr and suggest...,"[[tens, O], [years, O], [ago, O], [i, O], [con...",[],"[[0, 1, 2, 3], [5, 6, 7, 8, 9], [11, 12, 13], ..."
3,[],the parallels between the anc and the sicilian...,"[[the, O], [parallels, O], [between, O], [the,...",[],"[[0, 1, 2], [4, 5, 6, 7, 8, 9, 10, 11, 12], [1..."
4,[],intel community how can we work for a presid...,"[[intel, O], [community, O], [how, O], [can, O...",[],"[[0, 1, 2, 3, 4], [6, 7, 8, 9, 10, 11, 12, 13,..."
5,"[129, 130, 131, 132, 133, 134]",i live in an area that saw major flood damage ...,"[[i, O], [live, O], [in, O], [an, O], [area, O...",[stupid],"[[0], [2, 3, 4, 5], [7, 8], [10, 11], [13, 14,..."
6,"[35, 36, 37, 38, 39, 40, 41, 42, 43]",elaine marie jeffers sounds like a sociopath,"[[elaine, O], [marie, O], [jeffers, O], [sound...",[sociopath],"[[0, 1, 2, 3, 4, 5], [7, 8, 9, 10, 11], [13, 1..."
7,"[55, 56, 57, 58, 59, 60, 61, 62, 114, 115, 116...",a rock has more mental horsepower than the ind...,"[[a, O], [rock, O], [has, O], [more, O], [ment...","[ignorant, dishonest people]","[[0], [2, 3, 4, 5], [7, 8, 9], [11, 12, 13, 14..."
8,"[37, 38, 39, 40, 41, 158, 159, 160, 161, 204, ...",ha ha hillary lost all you hillary fools nee...,"[[ha, O], [ha, O], [hillary, O], [lost, O], [a...","[fools, dump, trash]","[[0, 1], [3, 4], [7, 8, 9, 10, 11, 12, 13], [1..."
9,"[94, 95, 96, 97]",yes it is to be expected oddly enough it...,"[[yes, O], [it, O], [is, O], [to, O], [be, O],...",[dumb],"[[0, 1, 2], [7, 8], [10, 11], [13, 14], [16, 1..."


## Word Embedding

In [1]:
import gensim
import gensim.downloader as gensim_api

In [2]:
nlp = gensim_api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


In [12]:
# corpus = tsd_train["text"]

corpus = [' hello world ']

lst_corpus = []
for index, row in tsd_train.iterrows():
    lst_tmp = []
    for each in row['sequence']:
        lst_tmp.append(each[0])
    lst_corpus.append(lst_tmp)

print(lst_corpus[:10])

[['another', 'violent', 'and', 'aggressive', 'immigrant', 'killing', 'a', 'innocent', 'and', 'intelligent', 'us', 'citizen', 'sarcasm'], ['i', 'am', '56', 'years', 'old', 'i', 'am', 'not', 'your', 'fucking', 'junior', 'pal', 'what', 'you', 'are', 'saying', 'makes', 'no', 'sense', 'i', 'don', 't', 'know', 'what', 'you', 'are', 'basing', 'this', 'on', 'the', 'cheap', 'black', 'market', 'crap', 'is', 'still', 'coming', 'up', 'from', 'mexican', 'nationals', 'and', 'the', 'good', 'stuff', 'from', 'bc', 'and', 'humbolt', 'the', 'i5', 'corridor', 'is', 'still', 'just', 'as', 'busy', 'and', 'they', 'are', 'still', 'making', 'a', 'ton', 'of', 'money', 'nothing', 'has', 'really', 'changed', 'except', 'that', 'the', 'legal', 'market', 'has', 'made', 'it', 'easy', 'for', 'anyone', 'of', 'legal', 'age', 'to', 'buy', 'it', 'and', 'the', 'legal', 'market', 'is', 'controlled', 'by', 'corporate', 'growers', 'with', 'million', 'dollar', 'facilities', 'or', 'a', 'co', 'op', 'of', 'smaller', 'growers', 't